<a href="https://colab.research.google.com/github/DomizianoScarcelli/big-data-project/blob/main/user_based_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Config

# Configuration

In [119]:
#@title Download necessary libraries
!pip install pyspark
!pip install -U -q PyDrive 
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u372-ga~us1-0ubuntu1~20.04).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [120]:
#@title Imports
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly

import pyspark
from pyspark.sql import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, FloatType, LongType
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.ml.linalg import SparseVector, DenseVector

from tqdm.notebook import tqdm
import time
import gc

from google.colab import drive

In [121]:
#@title Set up variables
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
GDRIVE_DIR = "/content/drive"
GDRIVE_HOME_DIR = GDRIVE_DIR + "/MyDrive"
GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/Big Data/datasets"
DATASET_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_friendly_spotify_playlist_dataset")
AUDIO_FEATURES_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_track_features")
LITTLE_SLICE_FILE = os.path.join(GDRIVE_DATA_DIR, "little_slice")
SMALL_SLICE_FLIE = os.path.join(GDRIVE_DATA_DIR, "small_slice")
LITTLE_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "little_slice_audio_features")
MICRO_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "micro_slice_audio_features")
SPLITTED_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "splitted_pyspark_track_features")
SAVED_DFS_PATH = os.path.join(GDRIVE_DATA_DIR, "saved_dfs")
RANDOM_SEED = 42 # for reproducibility
os.environ["JAVA_HOME"] = JAVA_HOME
os.environ["PYSPARK_PYTHON"]="python"

In [122]:
#@title Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '100G').\
                set("spark.executor.extraJavaOptions", "-XX:+UseG1GC").\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

ValueError: ignored

In [123]:
drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


## Setup ngrok

In [124]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [125]:
!ngrok authtoken 2NVN8kdoOnMVtlDGGWtwsbT5M3Q_2EJv2HE77FEXkz978Qtnq

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [126]:
from pyngrok import ngrok

# Open a ngrok tunnel on the port 4050 where Spark is running
port = '4050'
public_url = ngrok.connect(port).public_url

In [127]:
print("To access the Spark Web UI console, please click on the following link to the ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

To access the Spark Web UI console, please click on the following link to the ngrok tunnel "https://0735-35-196-5-69.ngrok-free.app" -> "http://127.0.0.1:4050"


In [128]:
#@title Check if everything is ok
spark, sc._conf.getAll()


(<pyspark.sql.session.SparkSession at 0x7f7f67ec2bc0>,
 [('spark.executor.extraJavaOptions',
   '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false -XX:+UseG1GC'),
  ('spark.driver.host', 'b11df5b7bf9d'),
  ('spark.app.name', 'PySparkTutorial'),


# Data acquisition

In [129]:

song_schema = StructType([
    StructField("pos", IntegerType(), True),
    StructField("artist_name", StringType(), True),
    StructField("track_uri", StringType(), True),
    StructField("artist_uri", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("album_uri", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("album_name", StringType(), True)
])

playlist_schema = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", ArrayType(song_schema), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

audio_features_schema = StructType([
    StructField("danceability", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("key", IntegerType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", IntegerType(), True),
    StructField("speechiness", FloatType(), True),
    StructField("acousticness", FloatType(), True),
    StructField("instrumentalness", FloatType(), True),
    StructField("liveness", FloatType(), True),
    StructField("valence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("type", StringType(), True),
    StructField("id", StringType(), True),
    StructField("uri", StringType(), True),
    StructField("track_href", StringType(), True),
    StructField("analysis_url", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("time_signature", IntegerType(), True)
])


In [130]:
playlist_df = spark.read.schema(playlist_schema).json(DATASET_FILE, multiLine=True)
slice_df = spark.read.schema(playlist_schema).json(SMALL_SLICE_FLIE, multiLine=True)
# slice_df = spark.read.schema(playlist_schema).json(LITTLE_SLICE_FILE, multiLine=True)
audio_df = spark.read.schema(audio_features_schema).json(SPLITTED_SLICE_AUDIO_FEATURES, multiLine=True) #has less songs than expected

In [131]:
# slice_df.select("tracks").first()

In [132]:
# slice_df.show()

# User-Based Collaborative Filtering
Note: The users are the playlists, the items are the songs and the ratings are 0 if the song is not in the playlist, 1 otherwise.

We have to define a function $sim(u,v)$ that defines the similarity between two users based on their ratings.

We represent the ratings $r_u \in \mathbb{R}^n$ as the $n$ dimensional vector that represents the ratings of the user $u$, where $n$ is the number of total songs in the dataset.

As the similarity function we can use Jaccard similarity.
\begin{equation}
sim(u,v) = J(r_u, r_v) = \frac{|r_u \cap r_v|}{|r_u \cup r_v|}
\end{equation}

Jaccard similarity ignores rating values, but we don't care here since the ratings are binary. In case of discrete value ratings we can use cosine similarity, or better pearson's correlation.

Done that, and defined as ${U^k}$ the neighborhood of $u$ ($k$ most similar users to $u$), we define the set of items rated by $u$'s neighborhood as

\begin{equation}
I^k = \{i \in I : \mathbf{r_{u,i}} \downarrow \land u \in U^k\}
\end{equation}

The rating for the item $i$ to the user $u$ will just be $\mathbf{r_u[i]}$.

In [133]:
from functools import wraps
import time

DEBUG = True
def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__} Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

In [134]:
RATING_VECTOR_FILE_PATH = os.path.join(SAVED_DFS_PATH, "playlist_rating_df.parquet")
PLAYLIST_EMBEDDINGS = os.path.join(SAVED_DFS_PATH, "playlist_embeddings.parquet")
FULL_PLAYLIST_EMBEDDINGS = os.path.join(SAVED_DFS_PATH, "full_playlist_embeddings.parquet")

In [135]:
def dense_to_sparse(dense: DenseVector) -> SparseVector:
  nonzero_indices = np.nonzero(np.array(dense))[0]
  nonzero_values = np.array(dense)[nonzero_indices]
  sparse_vector = SparseVector(len(dense), nonzero_indices.tolist(), nonzero_values.tolist())
  return sparse_vector

In [136]:
def get_all_songs(playlist_df: DataFrame, set_in_playlist: bool = False) -> DataFrame:
   all_songs = playlist_df.select(explode("tracks.track_uri").alias("track_uri")).distinct()
   if set_in_playlist:
     all_songs = all_songs.withColumn("in_playlist", lit(1))
   return all_songs
  
def get_songs_info(playlist_df: DataFrame, set_in_playlist: bool = False) -> DataFrame:
   all_songs = playlist_df.select(explode("tracks")).select("col.*").drop("pos").distinct()
   if set_in_playlist:
     all_songs = all_songs.withColumn("in_playlist", lit(1))
   return all_songs

In [137]:
songs_info_df = get_songs_info(slice_df)
songs_info_df.createOrReplaceTempView("SONGS_INFO")

songs_info_df = spark.sql("""
SELECT 
    row_number() OVER (
        PARTITION BY '' 
        ORDER BY '' 
    ) as pos,
    *
FROM 
    SONGS_INFO
""")

songs_info_df = songs_info_df.sort("track_uri")

songs_df = songs_info_df.select("pos", "track_uri")

RATING_VECTOR_LENGTH = songs_df.count()

In [138]:
# songs_info_df.show()
# songs_df.show()

Preprocessing the dataframe in order to associate to each track_uri an integer, that will represent the position of the track in the rating_vector. This is useful in order to avoid doing a lot of joins when generating the rating_vectors.

In [139]:
# def map_track_df_to_pos(playlist_df: DataFrame, mapping: DataFrame) -> List[DataFrame]:
#   songs_df_list = [get_all_songs(spark.createDataFrame([row])) for row in tqdm(slice_df.collect(), desc="Creating list of dataframes")]
#   track_uri_to_id = songs_df.select('track_uri', 'pos').rdd.collectAsMap()
#   track_uri_to_id_udf = udf(lambda x: track_uri_to_id.get(x), IntegerType())
#   songs_df_mapped_list = []

#   for df in tqdm(songs_df_list, desc="Mapping uris to pos"):
#       df = df.withColumn('pos', track_uri_to_id_udf(col('track_uri')))
#       songs_df_mapped_list.append(df)
  
#   return songs_df_mapped_list

# songs_df_mapped_list = map_track_df_to_pos(slice_df, songs_df)

In [140]:
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType, ArrayType

@timeit
def map_track_df_to_pos(playlist_df: DataFrame, mapping: DataFrame) -> DataFrame:
    track_uri_to_id = songs_df.select('track_uri', 'pos').rdd.collectAsMap()

    track_uri_to_id_udf = udf(lambda x: track_uri_to_id.get(x), IntegerType())

    # Define a UDF to map track URIs to integer IDs within each element of the list
    
    def extract_vector(tracks):
      pos_list = set()
      for row in tracks:
        uri = track_uri_to_id.get(row.track_uri)
        pos_list.add(uri)

      return SparseVector(RATING_VECTOR_LENGTH + 1, sorted(list(pos_list)), [1 for _ in pos_list])

    map_track_uri_udf = udf(lambda tracks: extract_vector(tracks), returnType=VectorUDT())

    # Apply the mapping UDF on the "tracks" column of the slice_df dataframe
    mapped_df = playlist_df.withColumn('tracks', map_track_uri_udf(col('tracks')))

    return mapped_df

if not os.path.exists(PLAYLIST_EMBEDDINGS):
  mapped_slice_df = map_track_df_to_pos(slice_df, songs_df)
  mapped_slice_df.write.parquet(PLAYLIST_EMBEDDINGS)
else:
  mapped_slice_df = spark.read.parquet(PLAYLIST_EMBEDDINGS)


In [141]:
#@title Old way of creating the rating vector dataframe
# def _create_rating_df(playlist_row: Row, songs_df: DataFrame) -> DataFrame:
#   """
#   Creates a dataframe that represents the "ratings" for a playlist in the dataframe
#   """
#   playlist_row = spark.createDataFrame([playlist_row], playlist_schema)
#   playlist_uris = get_all_songs(playlist_row)

#   joined = songs_df.join(playlist_uris, on="track_uri", how="right")
#   return joined


# def _check_songs_ordering(playlist_row: DataFrame, songs_df: DataFrame) -> bool:
#   """
#   Returns a boolean that indicates if the ordering in the songs_df and rating_df is the same
#   """
#   playlist_row = spark.createDataFrame([playlist_row], playlist_schema)
#   playlist_uris = get_all_songs(playlist_row, True).withColumnRenamed("in_playlist", "isin")

#   joined = songs_df.join(playlist_uris, on="track_uri", how="right")
#   joined_left = songs_df.join(playlist_uris, on="track_uri", how="left").filter("isin == 1")
#   assert joined.collect() == joined_left.collect(), f"The order of songs_df is different from the order of rating_df!"

# # def _extract_rating_vector(rating_df: DataFrame) -> SparseVector:
# #   """
# #   Extracts the rating vectors for each playlist 
# #   """
# #   dense_vector = DenseVector([row.isin for row in rating_df.select("isin").collect()])
# #   return dense_to_sparse(dense_vector)

# def _extrac_sparse_rating_vector(rating_df: DataFrame) -> SparseVector:
#   indices = np.sort([row.pos for row in rating_df.collect()])
#   return SparseVector(RATING_VECTOR_LENGTH, indices, np.ones(indices.shape[0]) )

# def rating_vector_from_row(playlist_row: Row, songs_df: DataFrame):
#   """
#   Pipelines togheter create_rating_df and extract_rating_vector.
#   """
#   rating_df_1 = _create_rating_df(playlist_row, songs_df)
#   rating_vector_1 = _extrac_sparse_rating_vector(rating_df_1)
#   return rating_vector_1

# # t1 = time.time() 

# # rating_vector_1 = rating_vector_from_row(slice_df.first(), songs_df)

# # t2 = time.time()

# # t2 - t1, rating_vector_1, type(rating_vector_1)

In [142]:
def jaccard_similarity(vector_1: SparseVector, vector_2: SparseVector) -> float:
  """
  Computes the Jaccard Similarity between two sparse binary vectors
  """
  # Convert SparseVectors to sets
  set1 = set(vector_1.indices)
  set2 = set(vector_2.indices)

  # Calculate the intersection and union of the sets
  intersection = len(set1.intersection(set2))
  union = len(set1.union(set2))

  # Calculate the similarity
  similarity = intersection / union

  return similarity

In [143]:
#@title Old way of creating the ratinf vector dataframe pt.2
# def create_rating_vectors_df(playlists_df: DataFrame) -> DataFrame:
#   rating_vectors = []

#   for playlist_row in tqdm(playlists_df.collect(), desc="Creating rating vectors"):
#     rating_vector = rating_vector_from_row(playlist_row, songs_df)
#     new_row = Row(playlist_id=playlist_row.pid, rating_vector=rating_vector)
#     rating_vectors.append([new_row])
#   return spark.createDataFrame(rating_vectors)

# if os.path.exists(RATING_VECTOR_FILE_PATH):
#   # rv_schema = StructType([StructField('playlist_id', LongType(), True), StructField('rating_vector', pyspark.ml.linalg.VectorUDT(), True)])
#   rating_vectors_df = spark.read.parquet(RATING_VECTOR_FILE_PATH)
#   rv_df = rating_vectors_df.select(col("_1.playlist_id").alias("playlist_id"), col("_1.rating_vector").alias("rating_vector"))
# else:
#   rating_vectors_df = create_rating_vectors_df(slice_df)
#   rating_vectors_df.write.parquet(RATING_VECTOR_FILE_PATH)

Creating a function that gets in input the playlist to continue, and returns a Dataframe that indicates its similarity with each other playlist in the dataset.

In [144]:
# mapped_slice_df.first()

In [145]:
@timeit
def create_similarity_df(input_vector: DataFrame, rating_vectors_df: DataFrame, similarityFunction: Callable) -> DataFrame:
  rv_df_input = rating_vectors_df.crossJoin(input_vector)
  similarity_udf = udf(similarityFunction, returnType='double')
  result_df = rv_df_input.withColumn("similarity", similarity_udf(rv_df_input["input_vector"], rv_df_input["rating_vector"]))
  return result_df

if DEBUG:
  rv_df = mapped_slice_df.withColumnRenamed("tracks", "rating_vector")
  # TODO: Just for test, we take the first playlist as the playlist to be continued 
  first_playlist_vector = rv_df.limit(1).select("rating_vector").withColumnRenamed("rating_vector","input_vector")
  result_df = create_similarity_df(first_playlist_vector, rv_df, jaccard_similarity)

Function create_similarity_df Took 0.1353 seconds


Curse of dimensionality! We can see that each playlist is very dissimilar from each other playlist.

If we filter the playlists that have a strictly positive similarity with the input playlist, and order them by descending similarity, we can see that the name (that we assume is very informative for the content of the playlist) is very similar, meaning that the algorithm seems to work!

In [146]:
# result_df.filter("similarity > 0").orderBy(col("similarity").desc()).show()

Now, in order to suggest some songs to continuate the input playlist, let's take the $k$ top most similar playlists

In [147]:
@timeit
def get_top_k_results(playlist_pid: int, similarity_df: DataFrame, k: int = 20) -> DataFrame:
  return similarity_df.filter( (col('pid') != playlist_pid)).orderBy(col("similarity").desc()).limit(k)

if DEBUG:
  first_playlist_pid = rv_df.limit(1).select("pid").first().pid
  top_k_results = get_top_k_results(first_playlist_pid, result_df)

Function get_top_k_results Took 0.0207 seconds


In [148]:
# top_k_results.show()

In [149]:
# from pyspark.ml.linalg import VectorUDT

# def add_sparse_vectors(accumulator: SparseVector, vector: SparseVector, weight: float) -> SparseVector:
#     accumulator_vec = accumulator.toArray() 
#     array_2 = vector.toArray() * weight

#     summed_array = accumulator_vec + array_2

#     values = [value for value in summed_array if value != 0]
#     sorted_indices = [index for index, value in enumerate(summed_array) if value != 0]
#     return SparseVector(accumulator_vec.size, sorted_indices, values)

# @udf(returnType=VectorUDT())
# def accumulate_sparse_vectors(accumulator, rating_vector, similarity):
#     summed_vector = add_sparse_vectors(accumulator, rating_vector, similarity)
#     return summed_vector

# df = top_k_results.withColumn('accumulated_vector', accumulate_sparse_vectors(top_k_results["rating_vector"], top_k_results["rating_vector"], top_k_results['similarity']))

We want to obtain a single embedding for all the $K$ top most similar playlists, that will be the rating vector. We can then pick the indices of the $n$ top greatest values form this vector, and those will be the $n$ songs that we will reccomend.

In order to aggregate the $k$ embeddings into a single one, I decided to take an average, weighted by the similarity value.

In [195]:
## @timeit
# def accumulate_top_k_results_2(top_k_results: DataFrame):
#   accumulator = np.zeros(RATING_VECTOR_LENGTH + 1)
#   similarity_sum = 0
#   #TODO: maybe do it with a rdd.map to be more performant
#   top_k = top_k_results.collect()
#   for row in top_k:
#     accumulator += row.rating_vector.toArray() * row.similarity
#     similarity_sum += row.similarity
  
#   accumulator /= similarity_sum
#   values = [value for value in accumulator if value != 0]
#   sorted_indices = [index for index, value in enumerate(accumulator) if value != 0]
#   return SparseVector(accumulator.size, sorted_indices, values)

@timeit
def get_input_rating_vector(similarity_df: DataFrame) -> SparseVector:
  return similarity_df.limit(1).select("input_vector").collect()[0].input_vector

@timeit
def accumulate_top_k_results(top_k_results: DataFrame, similarity_df: DataFrame) -> SparseVector:
    accumulator: np.ndarray = top_k_results.rdd.map(lambda row: (row.rating_vector.toArray() * row.similarity)).reduce(lambda x, y: (x+y))
    similarity_sum: float = top_k_results.rdd.map(lambda row: (row.similarity)).reduce(lambda x, y: x + y)

    accumulator /= similarity_sum

    #If a song is present in the input playlist, don't consider it
    input_indexes: SparseVector = get_input_rating_vector(similarity_df).indices

    values: List[float] = []
    sorted_indices: List[int] = []
    for index, value in enumerate(accumulator):
      if value != 0 and index not in input_indexes:
        values.append(value)
        sorted_indices.append(index)
    
    return SparseVector(len(accumulator), sorted_indices, values)

if DEBUG:
    accumulated_vector = accumulate_top_k_results(top_k_results, result_df)

Function get_input_rating_vector Took 0.3377 seconds
Function accumulate_top_k_results Took 1.7055 seconds


In [151]:
# @udf(returnType="int")
# def compute_vector_len(rating_vector):
#     return len(rating_vector.indices)
# final_df = df.withColumn('accumulated_vector_len', compute_vector_len(df["rating_vector"]))

In [152]:
# accumulated_vector = final_df.orderBy(col("accumulated_vector_len").desc()).first().accumulated_vector

In [236]:
@timeit
def get_top_n_values(vector: SparseVector, n:int = 10) -> List[Tuple[int, float]]:
  elements = list(enumerate(vector.toArray()))
  sorted_elements = sorted(elements, key=lambda x: x[1], reverse=True)
  top_n_indices = [(index, confidence) for index, confidence in sorted_elements[:n]]
  # return top_n_indices
  return [(x, y.item()) for (x, y) in top_n_indices] #Avoid the np.float64 type

if DEBUG:
  top_n_reccomendations = get_top_n_values(accumulated_vector)

Function get_top_n_values Took 0.2582 seconds


In [237]:
top_n_reccomendations
# [(364737, 0.866605950361276),
#  (18322, 0.8485493061139573),
#  (343633, 0.8485493061139573),
#  (373510, 0.8485493061139573),
#  (344041, 0.84513808582077),
#  (36962, 0.8245530907321501),
#  (377909, 0.8108159417687045),
#  (21652, 0.8095907772059491),
#  (10958, 0.7976130554051545),
#  (344569, 0.7976130554051545)]

[(364737, 0.866605950361276),
 (344041, 0.84513808582077),
 (36962, 0.8245530907321501),
 (377909, 0.8108159417687045),
 (27363, 0.7881661492818041),
 (375883, 0.7713627645609462),
 (372725, 0.7507684016253126),
 (23254, 0.7322990572091387),
 (15520, 0.7185418125956151),
 (368735, 0.711254956608903)]

In [198]:
@timeit
def convert_df_to_map(df: DataFrame) -> DataFrame:
  return df\
  .withColumn('aggregated', \
              to_json(
                  struct(
                      "artist_name", 
                      "track_uri", 
                      "artist_uri", 
                      "track_name", 
                      "album_uri", 
                      "duration_ms", 
                      "album_name")
                  )
              ).select("pos", "aggregated")
if DEBUG:
  songs_info_map = convert_df_to_map(songs_info_df)

Function convert_df_to_map Took 0.0500 seconds


In [227]:
prediction_schema = StructType([
     StructField('artist_name', StringType(), True),
     StructField('track_uri', StringType(), True),
     StructField('artist_uri', StringType(), True),
     StructField('track_name', StringType(), True),
     StructField('album_uri', StringType(), True),
     StructField('duration_ms', LongType(), True),
     StructField('album_name', StringType(), True),
    StructField('confidence', FloatType(), True),
     ])

reccomendations_schema = StructType([
     StructField('pos', IntegerType(), True),
     StructField('confidence', FloatType(), True),
     ])

In [238]:
import json
#TODO: For now this works, but it's very slow, and since this has to be executed online,
# consider to directly embed the song information inside the dataframe when computing the songs
# to recommend.
@timeit
def recommendation_song_info(recommendation: List[Tuple[int, float]], songs_info_map) -> Row:
  # result = []
  # for index, confidence in tqdm(recommendation, desc="Extracting song info from recommendations..."):
  #   song_info_dict = json.loads(songs_info_map.lookup(index)[0])
  #   song_info_dict.update({"confidence":float(confidence)})
  #   result.append(Row(**song_info_dict))
  # return spark.createDataFrame(result, schema=prediction_schema)
  top_n_reccomendations_df = spark.createDataFrame(top_n_reccomendations, schema=reccomendations_schema)
  return top_n_reccomendations_df.join(songs_info_df, "pos")

if DEBUG:
  songs_info = recommendation_song_info(top_n_reccomendations, songs_info_map.rdd)

Function recommendation_song_info Took 0.0601 seconds


### Putting it all togheter
We now define a single function that will get a playlist in input and will reccomend $n$ songs.

In [169]:
#TODO: this now takes a playlist and extracts its PID, if a playlist is built from scratch the PID shouldn't be defined
# A solution would be to pass the playlist row with the PID = Nan and then have a condition when extracting the PID. If Nan, ignore it
@timeit
def user_based_recommendation(playlist: DataFrame, mapped_slice_df: DataFrame, similarity_function: Callable, n:int = 50) -> DataFrame:
  rv_df = mapped_slice_df.withColumnRenamed("tracks", "rating_vector") #TODO: Parse the rv_df before and then remove this
  
  #TODO: define the songs_df as input to the function
  playlist_vector = map_track_df_to_pos(playlist, songs_df).select("tracks").withColumnRenamed("tracks", "input_vector")
  similarity_df = create_similarity_df(playlist_vector, rv_df, jaccard_similarity)

  top_k_results = get_top_k_results(playlist.first().pid, similarity_df)

  accumulated_vector = accumulate_top_k_results(top_k_results, similarity_df)

  top_n_indices = get_top_n_values(accumulated_vector, n=n)

  recommended_songs_info = recommendation_song_info(top_n_indices, songs_info_map.rdd)

  return recommended_songs_info

if DEBUG:
  #Collect and createDataFrame because operations on limit(1) take as long as the entire slice_df, don't know why
  playlist = spark.createDataFrame(slice_df.filter("pid == 1010").limit(1).collect())
  final_recommendation = user_based_recommendation(playlist, mapped_slice_df, jaccard_similarity, n=5)

## Performance Evaluation

In [ ]:
from sklearn.model_selection import train_test_split as sklearn_split

def train_test_split(playlist: Row) -> Tuple[Row, Row]:
    # SPLIT_RATIO = 0.8

    train_rows, test_rows = sklearn_split(playlist.tracks, random_state=42)

    playlist_train =  Row(
            name=playlist.name,
            collaborative=playlist.collaborative,
            pid=playlist.pid,
            modified_at=playlist.modified_at,
            num_tracks=playlist.num_tracks,
            num_albums=playlist.num_albums,
            num_followers=playlist.num_followers,
            tracks=train_rows,
            num_edits=playlist.num_edits,
            duration_ms=playlist.duration_ms,
            num_artists=playlist.num_artists,
        )

    playlist_test = Row(
            name=playlist.name,
            collaborative=playlist.collaborative,
            pid=playlist.pid,
            modified_at=playlist.modified_at,
            num_tracks=playlist.num_tracks,
            num_albums=playlist.num_albums,
            num_followers=playlist.num_followers,
            tracks=test_rows,
            num_edits=playlist.num_edits,
            duration_ms=playlist.duration_ms,
            num_artists=playlist.num_artists,
        )
    
    return playlist_train, playlist_test

# import time

# playlist = slice_df.filter(f"pid == {1000}").first()
# t0 = time.time()
# train, test = train_test_split(playlist)
# t1 = time.time()
# print(t1-t0)

Let's divide the whole playlist dataset into train and test splits

In [160]:
from pyspark.sql.functions import udf, struct
import shutil

def divide_whole_dataset(playlist_df: DataFrame) -> Tuple[DataFrame, DataFrame]:
  # Create a UDF to apply the divide_row function to each row
  train_test_split_udf = udf(train_test_split, returnType=ArrayType(StructType(playlist_df.schema.fields)))

  # Apply the divide_row UDF to each row of the DataFrame
  divided_df = playlist_df.withColumn("divided", train_test_split_udf(struct(*playlist_df.columns)))


  # Split the divided column into two separate columns: train and test
  train_test_df = divided_df.select(col('divided').getItem(0).alias('train'), col('divided').getItem(1).alias('test'))

  # Split the train and test columns into separate DataFrames
  train_df = train_test_df.select("train.*")
  test_df = train_test_df.select("test.*")
  return train_df, test_df

TRAIN_DF_PATH = os.path.join(SAVED_DFS_PATH, "train_df.parquet")
TEST_DF_PATH = os.path.join(SAVED_DFS_PATH, "test_df.parquet")

if os.path.exists(TRAIN_DF_PATH) and os.path.exists(TEST_DF_PATH):
  train_df = spark.read.parquet(TRAIN_DF_PATH)
  test_df = spark.read.parquet(TEST_DF_PATH)
else:
  # In order to avoid [PATH_ALREADY_EXISTS] errors. 
  if os.path.exists(TRAIN_DF_PATH):
    shutil.rmtree(TRAIN_DF_PATH)
  if os.path.exists(TEST_DF_PATH):
    shutil.rmtree(TEST_DF_PATH)

  train_df, test_df = divide_whole_dataset(slice_df)
  train_df.write.parquet(TRAIN_DF_PATH)
  test_df.write.parquet(TEST_DF_PATH)

In [161]:
@timeit
def precision_at_k(recommendations, ground_truth):
    """
    Calculates precision at k for the recommendations.
    """
    recommended_relevant_tracks = recommendations.join(ground_truth, "track_uri").count()
    precision = recommended_relevant_tracks / float(recommendations.count())
    return precision

In [162]:
@timeit
def evaluate(pid: int) -> Tuple[DataFrame, float]:
    playlist_train = train_df.filter(f"pid == {pid}")
    playlist_test = test_df.filter(f"pid == {pid}")
    ground_truth = playlist_test.select(explode("tracks")).select("col.*")
    recommendations = user_based_recommendation(playlist_train, mapped_slice_df, jaccard_similarity, n=ground_truth.count())
    precision = precision_at_k(recommendations, ground_truth)
    return recommendations, precision

In [239]:
reccomendations, precision = evaluate(2300)

Function map_track_df_to_pos Took 68.3643 seconds
Function create_similarity_df Took 0.0379 seconds
Function get_top_k_results Took 0.0199 seconds
Function get_input_rating_vector Took 1.8852 seconds
Function accumulate_top_k_results Took 60.8434 seconds
Function get_top_n_values Took 0.1912 seconds
Function recommendation_song_info Took 0.0483 seconds
Function user_based_recommendation Took 130.7605 seconds
Function evaluate Took 256.6253 seconds


In [243]:
precision

0.0

# Fighting against the curse of dimensionality: Matrix Factorization

We want to define $\mathbf{x}_u \in \mathbb{R}^d$ $d$-dimensional vector that represents the user $u$, and $\mathbf{w}_i \in \mathbb{R}^d$ vector that represent the item $i$.

We then can estimate the rating of user $u$ for the item $i$ by computing
\begin{equation}
\hat{r}_{u, i}=\mathbf{x}_u^T \cdot \mathbf{w}_i=\sum_{j=1}^d x_{u, j} w_{j, i}
\end{equation}
Or, in matrix notation,

\begin{equation}
\underbrace{R}_{m \times n} =
\underbrace{X}_{m \times d}
\underbrace{W^T}_{d \times n}
\end{equation}

### How to learn $X$ and $W$
The matrix $R$ is partially known and filled with the observations inside the dataset $\mathcal{D}$. In order to learn the latent factor representations $X$ and $W$, we minimize the following loss function:
\begin{equation}
L(X, W)=\sum_{(u, i) \in \mathcal{D}}\underbrace{\left(r_{u, i}-\mathbf{x}_u^T \cdot \mathbf{w}_i\right)^2}_{\text{squared error term}}+\underbrace{\lambda\left(\sum_{u \in \mathcal{D}}\left\|\mathbf{x}_u\right\|^2+\sum_{i \in \mathcal{D}}\left\|\mathbf{w}_i\right\|^2\right)}_{\text{regularization term}}
\end{equation}

We can then minimize the loss using Stochastic Gradient Descent or Alternating Least Squares.

# Matrix Factorization
Generate a matrix Y where each column represent a playlist and each row represent a song, the (i,j) entry will be 1 if the playlist contains the song, 0 otherwise.

In [165]:
# Throw error in order to not execute the following code
raise ValueError()

ValueError: ignored

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.functions import explode
spark.conf.set("spark.sql.pivotMaxValues", 1000000)

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.functions import expr


In [ ]:
from pyspark.sql.functions import explode
import random
tracks_df = slice_df.select("pid", explode("tracks").alias("track")).select("pid", "track.track_uri")
tracks_df = tracks_df.withColumn("rating", lit(1))
# tracks_df = tracks_df.withColumn("rating", (rand() * 10 + 1).cast("integer"))

In [ ]:
tracks_df.show()

In [ ]:
# # Explode the tracks array column into multiple rows
# # tracks_df = slice_df.select("pid", explode("tracks").alias("track"))
# # tracks_df = slice_df.select("pid", "tracks", "tracks")
# tracks_df = slice_df.select("pid", explode("tracks").alias("track")).select("pid", "track.track_uri", "track.pos")

# # Select relevant columns and add a rating column with value 1
# playlist_track_df = tracks_df.withColumn("rating", lit(1))

# # Get distinct track_uri values and join with playlist_track_df
# all_tracks_df = slice_df.select(explode("tracks").alias("track")).select("track.track_uri").distinct()
# all_playlists_df = slice_df.select("pid").distinct()

# all_against_all = all_tracks_df.join(all_playlists_df).distinct()

# from pyspark.sql.functions import when, col

# # playlist_track_rating_df = playlist_track_df.join(all_against_all, ["pid", "track_uri"], "left_outer") \
# #     .withColumn("rating", when(col("pos").isNull(), 0).otherwise(1))

# playlist_track_rating_df = all_against_all.join(playlist_track_df, ["pid", "track_uri"], "left_outer") \
#     .withColumn("rating", when(col("pos").isNull(), 0).otherwise(1)) \
#     .drop("pos")


In [ ]:
playlist_track_rating_df = tracks_df.withColumn("song_id", dense_rank().over(Window.orderBy("track_uri")))

In [ ]:
playlist_track_rating_df.show(truncate=False)

In [ ]:
als = ALS(userCol="pid", itemCol="song_id", ratingCol="rating", nonnegative=True, coldStartStrategy="drop")

In [ ]:
from typing import Tuple
import random

def train_test_split(df: DataFrame, split_ratio: float, seed: Optional[int] = None) -> Tuple[DataFrame, DataFrame]:
  random.seed(seed)
  distinct_pids = df.select("pid").distinct().rdd.map(lambda x: x[0]).collect()
  random.shuffle(distinct_pids)
  split_index = int(len(distinct_pids) * split_ratio)
  train_pids = distinct_pids[:split_index]
  test_pids = distinct_pids[split_index:]
  train_df = df.filter(col("pid").isin(train_pids))
  test_df = df.filter(col("pid").isin(test_pids))
  return train_df, test_df



In [ ]:
training, test = playlist_track_rating_df.randomSplit([0.8, 0.2], seed=42)

In [ ]:
model = als.fit(training)

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.show()

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [ ]:
predictions.filter(col("prediction") != "NaN").count(), predictions.filter(col("prediction") == "NaN").count()

In [ ]:
rmse

In [ ]:
subset = playlist_track_rating_df.select("pid").distinct().limit(1)
subUserRecs = model.recommendForUserSubset(subset, 10)

In [ ]:
subset.show()

In [ ]:
subUserRecs.show(truncate=False)

In [ ]:
def song_name_from_id(song_id: int, reverse_lookup: DataFrame) -> str:
  return 
  
def interpretRecommendation(recommended_result: DataFrame) -> str:
  return

In [ ]:
userRecs = model.recommendForAllUsers(1).orderBy("recommendations")
userRecs.show(truncate=False)
userRecs.count()

In [ ]:
slice_df.filter(col("pid") == 1710).select(explode("tracks.track_name")).show()

In [ ]:
track_uris = playlist_track_rating_df.filter(col("song_id") == 588).select("track_uri")
track_uris.first()